<a href="https://colab.research.google.com/github/erodola/NumMeth-s2-2023/blob/main/esercizi/ex3/ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Benvenuti alla terza esercitazione di Metodi Numerici!

Oggi vedremo il concetto di regolarizzazione (*sparsità* e *regolarità* (smoothness)) in modo pratico verificando l'effetto su segniali definiti su diversi domini. Importiamo i requirements, definiamo alcune funzioni di supporto e scarichiamo i dati necessari:

In [1]:
import librosa
import numpy as np
import IPython
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy
from PIL import Image
from tqdm import tqdm

In [2]:
def plot(x, y, title=None):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot()
    ax.set_xlabel('t')
    ax.set_title(title)
    ax.plot(x, y, '-')
    plt.show()

In [ ]:
!wget https://github.com/erodola/NumMeth-s2-2023/raw/main/esercizi/ex3/drums.wav
!wget https://github.com/erodola/NumMeth-s2-2023/raw/main/esercizi/ex3/mountain.png

## 1. Regolarizzazione di Tikhonov

Riprendiamo il fitting di polinomi usando il metodo dei minimi quadrati. Supponiamo di trovarci nel caso in cui il numero di punti a disposizione è inferiore a quello del grado del polinomio. Per esempio se abbiamo $n=4$, il sistema è sotto-determinato nel caso in cui il numero di coppie di punti a nostra disposizione è inferiore a 5. 

In [ ]:
n = 4

x2 = np.array([0, 1.])[:, np.newaxis]
y2 = np.array([2., 6.])[:, np.newaxis]

plt.scatter(x2, y2)

Proviamo a trovare una soluzione $\theta$ usando il regolarizzatore di Tikhonov
$ \mathbf{\theta} = (\mathbf{X}^T \mathbf{X} + \alpha \mathbf{I})^{-1} \mathbf{X}^T \mathbf{y}$. Per questo esempio scegliamo $\alpha = 1$.

In [ ]:
X2 = np.concatenate((x2**4, x2**3, x2**2, x2**1, x2**0),1)
print(f"X2 = {X2}")

alpha = 1.
theta2 = np.linalg.inv(X2.T @ X2 + alpha * np.eye(5)) @ X2.T @ y2
print(f"theta = {theta2}")

In [ ]:
x2_new = np.linspace(-2, 2, 20)[:, np.newaxis]
X2_new = np.concatenate((x2_new**4, x2_new**3, x2_new**2, x2_new**1, x2_new**0),1)
y2_new = X2_new@theta2  
print(f"y2_new = {y2_new}")

In [ ]:
plt.scatter(x2_new, y2_new)
plt.scatter(x2, y2, color='r')
plt.legend(['Model', 'Data'])

**Esercizio 1 - Alpha sweep**

 1. Rifare l'esempio aggiungendo un data point. Confrontare la soluzione
 per lo stesso valore di $\alpha = 1$
 2. Provare a variare il valore di $\alpha$ usando i valori [0.1, 0.5, 10., 100.]. Plottare le diverse curve fittate. Come cambiano le componenti di $\theta$ all'aumentare di $\alpha$ (calcolare la norma delle diverse $\alpha$)? 
 3. BONUS: Cosa succede quando $\alpha \to 0$? Come si può interpretare in modo matematico questo fenomeno?

In [ ]:
n = 4
alpha = 1.

# SCRIVERE QUI SOTTO IL CODICE DELL'ESERCIZIO
################

# Aggiungiamo un punto a x

x3 = ... 
y3 = ... 


# Creiamo la matrice dei dati X3
X3 = ... 

# Applichiamo la formula della ridge regression
theta3 = ... 

# Valutiamo il nuovo modello in alcuni punti
x3_new = ...
X3_new = ...
y3_new = ... 

# Plottiamo il nuovo modello (e confrontiamo con il modello fittato con 2 punti)
# plt.scatter(x2_new, y2_new)
# plt.scatter(x2, y2, color='r')
# plt.legend(['Model', 'Data'])
# plt.xlim(-2, 2)
# plt.ylim(-15, 15)
# plt.title('Ridge regression n = 4 con 2 punti (alpha = 1)')

# plt.figure()
# plt.scatter(x3_new, y3_new)
# plt.scatter(x3, y3, color='r')
# plt.legend(['Model', 'Data'])
# plt.xlim(-2, 2)
# plt.ylim(-15, 15)
# plt.title('Ridge regression n = 4 con 3 punti (alpha = 1)')

# Effettuiamo la regressione regolarizzata per diversi valori di alpha
for alpha_i in [0.1, 0.5, 10., 100.]:
  theta = ... 
  # Stampiamo i valori di theta e la relativa norma
  print(...)
  print(...)
  
  # valutiamo il modello nei nuovi punti
  y_new = ... 
    
  # plt.figure()
  # plt.scatter(x3_new, y_new)
  # plt.scatter(x3, y3, color='r')
  # plt.legend(['Model', 'Data'])
  # plt.xlim(-2, 2)
  # plt.ylim(-15, 15)
  # plt.title(f"Ridge regression n = 4 con 3 punti (alpha = {alpha_i})")

## 2. Regolarizzazione per problemi 1D

### 2.1 Deblurring

Passiamo ad esaminare segnali uni-dimensionali. A tal fine carichiamo la traccia di batteria che abbiamo gia incontrato nella prima lezione e selezioniamo un chunk di 300 sample.

In [178]:
sr = 44100
# carichiamo 300 samples a partire da 0:10
t = 10
n = 300
drums, _ = librosa.load('drums.wav', sr=sr)
drums_chunk = drums[t * sr: t * sr + n][:, np.newaxis]

Visualizziamo il segnale:

In [ ]:
y_plot = drums_chunk
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale originale")

Possiamo sfocare la traccia applicando un *kernel Gaussiano* locale $K \in \mathbb{R}^{n\times n}$, tale che $K_{i, j} = g_{a,c}(j - i)$, dove $$g_{a,c}(j - i) = a \exp{-\frac{(j - i)^2}{2c^2}}.$$ Quando applichiamo un kernel Gaussiano a un segnale, stiamo effettivamente eseguendo una sorta di media ponderata locale dei valori del segnale. I valori del segnale vicino al centro del kernel hanno un peso maggiore nella media, mentre i valori più lontani dal centro hanno un peso minore.

È facile verificare che ogni diagonale di questa matrice contiene termini costanti. Una matrice che soddisfa questa proprietà è detta *matrice di Toeplitz*.

In [ ]:
kernel_interval = np.arange(-n//2, n//2)

a = 1.
c = 4.
kernel_row = a*np.exp(-kernel_interval**2/(2*c**2))

y_plot = kernel_row
x_plot = kernel_interval
plot(x_plot, y_plot, title="Riga kernel Gaussiano (a = 1, c = 4)")

In [ ]:
kernel_row_shifted = np.roll(kernel_row, shift=-n//2)
y_plot = kernel_row_shifted
x_plot = kernel_interval
plot(x_plot, y_plot, title="Riga kernel Gaussiano shiftata (a = 1, c = 4)")

In [ ]:
K = scipy.linalg.toeplitz(kernel_row_shifted)
plt.imshow(K)
plt.colorbar()

Possiamo sfocare il segnale semplicemente applicando la matrice $K$:

In [183]:
drums_chunk_blur_c1 = K @ drums_chunk

In [ ]:
y_plot = drums_chunk_blur_c1
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale sfocato ($a=1, c=4$)")


y_plot = drums_chunk
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale originale")

Osserviamo che il segnale è diventato più *smooth*. Vediamo cosa succede se aumentiamo il parametro $c$:

In [ ]:
a = 1.
c = 8.                                                            
kernel_row = a*np.exp(-kernel_interval**2/(2*c**2))
                                                                   
y_plot = kernel_row                                                
x_plot = kernel_interval                                           
plot(x_plot, y_plot, title="Kernel Gaussiano (a = 1, c = 8)")    

kernel_row_shifted = np.roll(kernel_row, shift=-n//2)
y_plot = kernel_row_shifted
x_plot = kernel_interval
plot(x_plot, y_plot, title="Kernel Gaussiano shiftato (a = 1, c = 8)")

K = scipy.linalg.toeplitz(kernel_row_shifted)
plt.imshow(K)
plt.colorbar()

In [186]:
drums_chunk_blur_c2 = K@drums_chunk

In [ ]:
y_plot = drums_chunk_blur_c2
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale sfocato ($a=1, c=8$)")

Ci poniamo il seguente problema inverso: conoscendo la traccia sfocata $x_{\text{blurry}}$ e l'operatore di blurring $G = K$, possiamo risalire alla traccia originale $x$? Il problema è sotto-determinato essendoci infinite $x$ tali che $G x = x_{\text{blurry}}$. A tal fine risolviamo il problema usando un regolarizzatore di Tikhonov, provando diversi valori di $\alpha$

In [188]:
x_blurry = drums_chunk_blur_c2

alpha = 0.01

x_1 = np.linalg.inv(K.T@K + alpha*np.eye(K.shape[1]))@K.T@x_blurry

In [ ]:
y_plot = x_1
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale ricostruito ($a=1$, $c=8$, $\\alpha = 0.01$)")

y_plot = drums_chunk_blur_c1
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale sfocato ($a=1, c=4$)")

y_plot = drums_chunk
x_plot = np.arange(y_plot.shape[0])
plot(x_plot, y_plot, title="Segnale originale")

Non siamo riusciti a ricostruire fedelmente i dettagli, ma confrontando la soluzione con il segnale affetto da blur con $c=4$, notiamo che ci siamo avvicinati al segnale originale. Per avere una ricostruzione fedele si possono introdurre tecniche di learning (che esulano dallo scope di questa lezione).

L'esempio precedente non può essere ascoltato. Questo perchè codifichiamo solamente 300/44100 = 0.006s di traccia, una quantità impercettibile all'udito. 

Ripetiamo l'esempio con 2 secondi di traccia ($n$ = 88200)

In [ ]:
# carichiamo 88200 samples a partire da 0:10
n = 88200
drums_long = drums[t * sr: t * sr + n][:, np.newaxis]

IPython.display.Audio(drums_long[:, 0], rate=sr)

Se proviamo a creare la matrice 𝐾 come prima ci imbattiamo in problemi di memoria (se provate a farlo il kernel Jupyter viene restartato avendo saturato la memoria). 

Nei casi in cui la maggior parte delle entrate di una matrice sono zeri, conviene usare matrici sparse. Le matrici sparse memorizzano solamente le entrate diverse da zero, e quindi è possibile rappresentare dati alto-dimensionali senza imbatersi in problemi di memoria. Dato che il kernel Gaussiano tende esponenzialmente a zero allontanandosi dalla media, le matrici di blurring in effetti possono essere rappresentate come matrici sparse.

In [ ]:
# Scegliamo un numero di diagonali non nulle nella matrice sparse (in questo caso 40).
nonzero_entries = 40


kernel_interval = np.arange(-nonzero_entries//2, nonzero_entries//2, dtype=np.int64) #entries, dtype=np.int64) - (entries//2)

a = 1.
c = 8.
kernel_row = a*np.exp(-kernel_interval**2/(2*c**2))
K = sparse.diags(kernel_row, kernel_interval, shape=(n, n)) 

print(f"K = {repr(K)}")

Se usassimo una matrice densa, $K$ peserebbe 88200 $\times$ 88200 $\times$ 8 bytes = 62 Gb 🤡. Dato che usiamo solo 3527600 elementi, quest'ultima pesa solamente 3 Mb, ~1764 volte di meno!

Smoothiamo il segnale con $K$:

In [ ]:
drums_long_blur = K @ drums_long

IPython.display.Audio(drums_long_blur[:, 0], rate=sr)

Per effettuare i minimi quadrati con matrici sparse, non possiamo procedere come nel caso denso. Esistono metodi iterativi (metodi che vedremo più approfonditamente nelle prossime lezioni) per minimi quadrati su matrici sparse. Usiamo il metodo *LSMR* (https://arxiv.org/abs/1006.0758) implementato in `scipy`. Il valore $\alpha$ della regolarizzazione di Tikhonov può essere impostato nel parametro `damp`:

In [ ]:
x = scipy.sparse.linalg.lsmr(K, drums_long_blur, damp=0.01)[0]
IPython.display.Audio(x, rate=sr)

### 2.2 Denoising

Possiamo affrontare il problema ortogonale al deblurring, ossia il denoising. Supponiamo che il nostro segnale $\mathbf{x}$ sia stato corrotto con del rumore Gaussiano $\mathbf{z} \sim \mathcal{N}(\mathbf{0}, \sigma^2\mathbf{I})$ ($\sigma$ ha un ruolo simile alla $c$ vista nell'esempio di deblurring, in effetti se la funzione Gaussiana dell'esempio precedente e' tale che il suo integrale su tutto $\mathbb{R}$ è 1 allora  $\sigma = c$; da notare che in questo caso abbiamo una Gaussiana multivariata): $$\mathbf{x}_{\text{noisy}} = \mathbf{x} + \mathbf{z}.$$ Il nostro task adesso è quello di ottenere una soluzione più smooth, rimuovendo il rumore. A tal fine vogliamo risolvere il seguente problema di minimizzazione:
$$ \mathbf{x}_{\text{smooth}} = \operatorname*{arg\,min}_\mathbf{x} \Vert \mathbf{x} - \mathbf{x}_{\text{noisy}}\Vert^2_2 + \alpha \Vert D \mathbf{x}\Vert^2_2 $$ dove $D$ è la derivata discreta: $$ D = \begin{bmatrix} 1 & -1 & \dots & 0 \\ 0 & 1 & \dots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \dots & 1 \end{bmatrix}.$$ Anche in questo caso la matrice $D$ è di Toepliz.


## Esercizio 2
1. Scrivere l'equazione normale per il problema di minimizzazione definito sopra.
2. Perturbare `drums_chunk` con rumore Gaussiano bianco per ottenere `x_noisy`. Clippare nell'intervallo -1 e 1.
3. Risolvere il problema dei minimi quadrati regolarizzato per trovare la soluzione e plottare la soluzione.
4. Perturbare tutto `drums_long`, ottenendo `drums_long_noisy` poi ascoltare la versione noisy. 
5. Risolvere il problema su `drums_long` applicando iterativamente il metodo su chunk locali. Ascoltare il risultato.


In [170]:
chunk_size = 300
sigma = 0.05
alpha = 100.
drums_small_chunk = drums_long[:n]
from scipy.linalg import toeplitz

# SCRIVERE QUI SOTTO IL CODICE DELL'ESERCIZIO
################


# Perturbiamo drums_small_chunk con rumore Gaussiano
# Campioniamo una z (usare np.random.randn e moltiplicare per sigma**2)

noise = ...
x_noisy = ...
# x_noisy = np.clip(x_noisy , -1, 1)


# Risolviamo per la x (usare np.diags poi convertire a np.ndarray)

# D = sparse.diags(...).toarray()
x_smooth = ...

# plt.figure()
# y_plot = ...
# x_plot = np.arange(y_plot.shape[0])
# plot(x_plot, y_plot, title="Segnale noisy ($\\sigma = 0.05$)")

# plt.figure()
# y_plot = ...
# x_plot = np.arange(y_plot.shape[0])
# plot(x_plot, y_plot, title="Segnale originale")

# plt.figure()
# y_plot = ...
# x_plot = np.arange(y_plot.shape[0])
# plot(x_plot, y_plot, title="Segnale ricostruito ($\\alpha = 100, \\sigma = 0.05$)")

In [171]:
# Risolviamo su drums_long

noise_long = ...
# drums_long_noisy = np.clip(..., -1, 1)

# IPython.display.Audio(drums_long_noisy[:, 0], rate=sr)

In [172]:
# smoothed_drums_long = np.zeros_like(drums_long_noisy)

# for i in range(88200 // n):
#   x_noisy_chunk = ...
#   x_smooth_chunk = ...
#   smoothed_drums_long[i*chunk_size:(i+1)*chunk_size] = x_smooth_chunk

# IPython.display.Audio(smoothed_drums_long[:, 0], rate=sr)

## 3. Regolarizzazione per problemi 2D

Passiamo a problemi di regolarizzazione nel setting 2D. A tal proposito, ripetiamo l'esercizio precedente nel setting 2D.

## Esercizio 3
1. Caricare l'immagine `mountain.jpg` usando PIL
2. Perturbare l'immagine con rumore Gaussiano bianco per ottenere `x_noisy`
3. Risolvere il problema di denoising applicando iterativamente il metodo dei minimi quadrati prima sulle righe poi sulle colonne. Visualizzare il risultato.

In [ ]:
sigma = 0.5
alpha = 6.
n = 256 
chunk_size = 32

# SCRIVERE QUI SOTTO IL CODICE DELL'ESERCIZIO
################


# carichiamo l'imagine con PIL
image = Image.open('mountain.png')
# trasformiamo in un vettore np dopo aver normalizzato
pix = np.array(image) / np.max(np.array(image))

# plottiamo l'immagine
plt.imshow(pix)

# creare rumore e aggiungere al dato
noise = ...
x_noisy = ...
# x_noisy = np.clip(x_noisy, 0., 1.)

# plottiamo l'immagine noisy
# plt.imshow(x_noisy)
# plt.show()

# kernel di derivazione 
def denoise(signal, alpha):
    # D = sparse.diags(...).toarray()
    x_smooth = ...
    return x_smooth
    
# iteriamo su ogni riga e canale
# denoised_image = np.zeros_like(x_noisy)
# for i in ...:
#   for c in range(...):
#     denoised_image[i, :, c] = ...

# plt.imshow(denoised_image)
# plt.show()

# iteriamo su ogni colonna
# denoised_image_2d = np.zeros_like(denoised_image)
# for i in ...:
#   for c in ...:
#    denoised_image_2d[:, i, c] = ...

# plt.imshow(denoised_image_2d)
# plt.show()